### Text'ler Üzerinden Özellik Türetmek

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
# !pip install missingno
import missingno as msno
from datetime import date
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)

In [2]:
def load():
    data = pd.read_csv("titanic.csv")
    return data
df = load()
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S


#### Letter Count : Harfleri Saydırma
* DİKKAT: Bir meta bilgi geldi aklımıza bunları bir saydıralım dedik. Öyle ya bazı isimler mesela kraliyet ailesinin soyundandır şeklinde bir isimlendirme olabilir, bunu tamamen örnek veröek için uydurduk. Belki kraliyet ailelerine yapılan bir ayrıcalık vardır ve o taa buraya yansımıştır. Bunlar senaryolardır.

In [4]:
df["NEW_NAME_COUNT"] = df["Name"].str.len()
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_NAME_COUNT
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S,23
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C,51
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S,22
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S,44
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S,24


#### Word Count: Kelimeleri Saydırma
* İlgili ismi yakaladığında bu ismi komple bir stringe çevirsin daha sonra bunu boşluklara göre split etsin. Kaç tane burada kelime varsa bunu len ile saysın.

In [5]:
df["NEW_NAME_WORD_COUNT"] = df["Name"].apply(lambda x: len(str(x).split(" ")))
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_NAME_COUNT,NEW_NAME_WORD_COUNT
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S,23,4
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C,51,7
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S,22,3
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S,44,7
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S,24,4


#### Özel Yapıları Yakalamak
* Konu anlatımı bölümünde bu textlerin içinde Dr gibi bazı özel isimler vardı. Bu Dr ismine sahip olanları eklesek/işaretlesek, bu doktor bu değil şeklinde bir çıkarım yapsak. Bildiğimiz üzere veri setinde mesleklerle ilgili bir değişkenimiz yoktu. Ama bizim elimizde çok değerli bir bakış açısı var. Öte yandan featureları sadeleştirme yorumumuz da gözden kaçmamalıdır. Mümkün olduğu kadar az sayıda değişkenle yüksek başarılara erişmeye çalışıyoruz. Bu yüzden rare encoding bölümünde elemeler yaptık Ama feature engineering bölümünde öyle değişkenler türeyebilir ki bunlar bir şekilde hataları düşürür, başarılı tahminler elde edilme imkanını sağlayabilir. Bu sebeple mümkün olduğu kadar kurcalıyoruz.

* Aşağıdaki kodda ilgili satırı split ediyoruz. Bunu yaptığımızda her bir değer burada tek başına bir liste halinde erişilebilir olacak. Sonra for x in ile bunlarda gezeceğiz. Eğer o gezdiğimiz her bir kelimenin başlangıcında Dr ifadesi varsa bunu seçeceğiz. len'ini alıyoruz ancak bu bir yorum başka şekilde de yapılabilir. Dolayısıyla Dr olanlarda 1 sayısı gelecektir.

In [7]:
df["NEW_NAME_DR"] = df["Name"].apply(lambda x: len([x for x in x.split() if x.startswith("Dr")]))
df.head()
# head'ine baktığımızda gözlemleyemiyor olacağız.

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_NAME_COUNT,NEW_NAME_WORD_COUNT,NEW_NAME_DR
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S,23,4,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C,51,7,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S,22,3,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S,44,7,0
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S,24,4,0


In [9]:
# head yerine bunu yaparsak gözlemleyebiliriz.
df.groupby("NEW_NAME_DR").agg({"Survived": "mean"})

,Survived
NEW_NAME_DR,
0,0.383
1,0.500


* DİKKAT: Doktor olanların hayatta kalma oranı daha yüksektir. Burada yine dikkat edilmesi gereken nokta: NEW_NAME_DR kategorik değişken ya şimdi, bunun frekansları ne bilmiyoruz. Bunu da bir gözlemlemeye çalışalım.

In [10]:
df.groupby("NEW_NAME_DR").agg({"Survived": ["mean","count"]})

Survived      
                mean count
NEW_NAME_DR               
0              0.383   881
1              0.500    10

* 900 tane gözlem birismi içerisinden 10 tane doktor olması önemli mi değil 